# Chemical equilibrium with fixed phase amount

**Problem**: consider an aqueous solution at 60 °C obtained by mixing 1 kg of H<sub>2</sub>O, 1 mol of NaCl and 1 mol of CO<sub>2</sub>. What must the pressure of this aqueous solution be so that it becomes saturated with CO<sub>2</sub> (forming bubbles of CO<sub>2</sub>)?

We formulate a chemical equilibrium problem to calculate the CO<sub>2</sub> bubble/saturation pressure P<sub>bubble</sub> in the brine. The chemical system consists of aqueous and gaseous phases. The **amount of gaseous phase must be constraint to be a tiny value** to model the limit condition in which a bubbles are formed.

In [1]:
from reaktoro import *

db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase(speciate("H O Na Cl C"))
solution.setActivityModel(ActivityModelPitzerHMW())

gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelPengRobinson())

system = ChemicalSystem(db, solution, gases)

The initial chemical state for this system represents the 1 molal NaCl and 1 molal CO<sub>2</sub> brine at 60 °C:

In [2]:
state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 1.0, "mol")

print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |    333.1500 |    K |
| Pressure        |      1.0000 |  bar |
| Charge:         | -1.0000e-16 |  mol |
| Element Amount: |             |      |
| :: H            |  1.1101e+02 |  mol |
| :: C            |  1.0000e+00 |  mol |
| :: O            |  5.7506e+01 |  mol |
| :: Na           |  1.0000e+00 |  mol |
| :: Cl           |  1.0000e+00 |  mol |
| Species Amount: |             |      |
| :: H+           |  1.0000e-16 |  mol |
| :: H2O          |  5.5506e+01 |  mol |
| :: CO3-2        |  1.0000e-16 |  mol |
| :: CO2          |  1.0000e+00 |  mol |
| :: Cl-          |  1.0000e+00 |  mol |
| :: HCO3-        |  1.0000e-16 |  mol |
| :: Na+          |  1.0000e+00 |  mol |
| :: OH-          |  1.0000e-16 |  mol |
| :: CO2(g)       |  1.0000e-16 |  mol |
| :: H2O(g)       |  1.0000e-16 |  mol |
+-----------------+-------------+------+


These constraint specifications are provided with the [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) and the desired values of 

* **temperature** and 
* **amount of gas phase** 

are set via [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html). We also define **lower and upper limits for pressure** (to avoid negative and unrealistic large pressure values during the equilibrium calculation):

In [3]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.phaseAmount("GaseousPhase")

solver = EquilibriumSolver(specs)

conditions = EquilibriumConditions(specs)
conditions.temperature(60.0, "celsius")
conditions.phaseAmount("GaseousPhase", 1.0, "umol")  # umol = 1e-6 moles
conditions.setLowerBoundPressure(1.0, "bar")
conditions.setUpperBoundPressure(1000.0, "bar")

solver.solve(state, conditions)

print("FINAL STATE")
print(state)

FINAL STATE
+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |    333.1500 |    K |
| Pressure        |    259.7338 |  bar |
| Charge:         | -1.4299e-17 |  mol |
| Element Amount: |             |      |
| :: H            |  1.1101e+02 |  mol |
| :: C            |  1.0000e+00 |  mol |
| :: O            |  5.7506e+01 |  mol |
| :: Na           |  1.0000e+00 |  mol |
| :: Cl           |  1.0000e+00 |  mol |
| Species Amount: |             |      |
| :: H+           |  1.2704e-03 |  mol |
| :: H2O          |  5.5505e+01 |  mol |
| :: CO3-2        |  6.6145e-10 |  mol |
| :: CO2          |  9.9873e-01 |  mol |
| :: Cl-          |  1.0000e+00 |  mol |
| :: HCO3-        |  1.2704e-03 |  mol |
| :: Na+          |  1.0000e+00 |  mol |
| :: OH-          |  1.8094e-10 |  mol |
| :: CO2(g)       |  9.7193e-07 |  mol |
| :: H2O(g)       |  2.8071e-08 |  mol |
+-----------------+-------------+------+


**TASK:** print the CO<sub>2</sub> saturation pressure for our aqueous solution.

In [4]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #
P = state.pressure() * 1e-5  # convert pressure from Pa to bar
print("Computed saturation pressure is", P, "bar")

Computed saturation pressure is 259.734 bar
